In [18]:
from os.path import join
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
dpath = 'data' # Data path

dat = pd.read_csv(join(dpath, '2022-09-14 keystroke detail.psv'), sep='|')
print(dat.info())
dat

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389560 entries, 0 to 389559
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   healthCode          389560 non-null  object 
 1   recordId            389560 non-null  object 
 2   createdOnTimestamp  389560 non-null  int64  
 3   createdOnTimeZone   389560 non-null  int64  
 4   appVersion          389560 non-null  object 
 5   phoneInfo           389560 non-null  object 
 6   sessionDuration     389560 non-null  float64
 7   sessionTimestamp    389560 non-null  int64  
 8   sessionTimeZone     389560 non-null  int64  
 9   sampleNumber        389560 non-null  int64  
 10  timestamp           389560 non-null  float64
 11  keypress_type       389560 non-null  object 
 12  keypress_duration   380520 non-null  float64
 13  <Unknown1>          380520 non-null  float64
 14  <Unknown2>          363158 non-null  float64
dtypes: float64(5), int64(5), object(5)

,healthCode,recordId,createdOnTimestamp,createdOnTimeZone,appVersion,phoneInfo,sessionDuration,sessionTimestamp,sessionTimeZone,sampleNumber,timestamp,keypress_type,keypress_duration,<Unknown1>,<Unknown2>
0,b4115696-6c7c-4893-abd8-4345a9696933,3C0j62Wt26aXH5bzJFgLESed,1654003809624,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,1]; iOS/15.4.1",21.129277,1654003809,0,1,1.654004e+09,alphanum,0.066594,14.676121,NaN
1,b4115696-6c7c-4893-abd8-4345a9696933,3C0j62Wt26aXH5bzJFgLESed,1654003809624,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,1]; iOS/15.4.1",21.129277,1654003809,0,2,1.654004e+09,alphanum,0.049791,9.617690,131.946527
2,b4115696-6c7c-4893-abd8-4345a9696933,3C0j62Wt26aXH5bzJFgLESed,1654003809624,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,1]; iOS/15.4.1",21.129277,1654003809,0,3,1.654004e+09,alphanum,0.058424,26.833856,110.852243
3,b4115696-6c7c-4893-abd8-4345a9696933,3C0j62Wt26aXH5bzJFgLESed,1654003809624,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,1]; iOS/15.4.1",21.129277,1654003809,0,4,1.654004e+09,alphanum,0.033104,14.864208,90.629895
4,b4115696-6c7c-4893-abd8-4345a9696933,3C0j62Wt26aXH5bzJFgLESed,1654003809624,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,1]; iOS/15.4.1",21.129277,1654003809,0,5,1.654004e+09,alphanum,0.066655,14.766706,172.011942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389555,b4115696-6c7c-4893-abd8-4345a9696933,vJw0dL-ldndpmMneyfQjZf8I,1663200405433,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,2]; iOS/14.8",7.663801,1663200405,0,4,1.663200e+09,alphanum,0.100029,6.222715,250.116861
389556,b4115696-6c7c-4893-abd8-4345a9696933,vJw0dL-ldndpmMneyfQjZf8I,1663200405433,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,2]; iOS/14.8",7.663801,1663200405,0,5,1.663200e+09,alphanum,0.108309,5.233434,202.866402
389557,b4115696-6c7c-4893-abd8-4345a9696933,vJw0dL-ldndpmMneyfQjZf8I,1663200405433,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,2]; iOS/14.8",7.663801,1663200405,0,6,1.663200e+09,alphanum,0.082735,9.902297,65.940214
389558,b4115696-6c7c-4893-abd8-4345a9696933,vJw0dL-ldndpmMneyfQjZf8I,1663200405433,-500,"version 2.2.0, build 30","Unknown iPhone [iPhone13,2]; iOS/14.8",7.663801,1663200405,0,7,1.663200e+09,alphanum,0.091890,13.672761,87.560141


In [31]:
# Aggregate into sessions based on timestamp
percentiles = [25, 50, 75, 90]
funcs = []
for p in percentiles:
    funcs.append(partial(np.nanpercentile, q = p))

ikds = dat.groupby('sessionTimestamp')['keypress_duration'].agg(funcs)
ikds.columns = ["IKD{}".format(p) for p in percentiles]
ikds.set_index([list(range(len(ikds)))], inplace=True)
ikds

c:\Users\lorkno\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\nanfunctions.py:1559: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,


,IKD25,IKD50,IKD75,IKD90
0,0.074860,0.082998,0.095410,0.103017
1,0.073959,0.082158,0.099411,0.109763
2,0.058271,0.074949,0.083600,0.099863
3,0.097808,0.103997,0.110192,0.113783
4,0.099803,0.107943,0.108564,0.118693
...,...,...,...,...
8440,0.083733,0.092151,0.108248,0.114298
8441,0.062737,0.074336,0.083211,0.093724
8442,0.065703,0.066777,0.083146,0.096279
8443,0.066297,0.082377,0.091807,0.106935
